<a href="https://colab.research.google.com/github/aiswarya-1422/ictak/blob/main/NLP_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


In [2]:
data = pd.read_csv('/content/Preprocessed_DonorsChoose_dataset.csv')

In [3]:
data.head()


,id,teacher_prefix,school_state,project_grade_category,project_subject_categories,project_subject_subcategories,teacher_number_of_previously_posted_projects,project_is_approved,price,quantity,cleaned_titles,cleaned_essays,cleaned_summary,isdigit_summary
0,p253737,mrs,in,grades_prek_2,literacy_language,esl_literacy,0,0,154.60,23,educational support english learners home,students english learners working english seco...,students_need_opportunities_practice_beginning...,0.0
1,p258326,mr,fl,grades_6_8,history_civics_health_sports,civics_government_teamsports,7,1,299.00,1,wanted projector hungry learners,students arrive school eager learn polite gene...,students_need_projector_help_viewing_education...,0.0
2,p182444,ms,az,grades_6_8,health_sports,health_wellness_teamsports,1,0,516.85,22,soccer equipment awesome middle school students,true champions not always ones win guts mia ha...,students_need_shine_guards_athletic_socks_socc...,0.0
3,p246581,mrs,ky,grades_prek_2,literacy_language_math_science,literacy_mathematics,4,1,232.90,4,techie kindergarteners,work unique school filled esl english second l...,students_need_engage_reading_math_way_inspire_...,0.0
4,p104768,mrs,tx,grades_prek_2,math_science,mathematics,1,1,67.98,4,interactive math tools,second grade classroom next year made around 2...,students_need_hands_practice_mathematics_fun_p...,0.0


In [4]:
class_counts = data['project_is_approved'].value_counts()
class_percent = data['project_is_approved'].value_counts(normalize=True) * 100

print("Class Counts:\n", class_counts)
print("\nClass Percentages:\n", class_percent)


Class Counts:
 project_is_approved
1    84391
0    15078
Name: count, dtype: int64

Class Percentages:
 project_is_approved
1    84.841508
0    15.158492
Name: proportion, dtype: float64


In [5]:
import string
import nltk
import re

In [6]:
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [8]:
def preprocess_data(text):
  if pd.isnull(text):
   return""
    # Lowercasing
  text = text.lower()
    # Remove HTML tags
  text = re.sub(r"<.*?>", " ", text)
    # Remove numbers
  text = re.sub(r"\d+", " ", text)
    # Remove punctuation
  text = text.translate(str.maketrans("", "", string.punctuation))
    # Keep only letters and spaces
  text = re.sub(r"[^a-z\s]", " ", text)
    # Tokenize
  tokens = text.split()
    # Remove stopwords
  tokens = [w for w in tokens if w not in stop_words]
    # Lemmatize
  tokens = [lemmatizer.lemmatize(w) for w in tokens]
    # Rejoin back to string
  return " ".join(tokens)

data['processed_essays'] =  data['cleaned_essays'].apply(preprocess_data)
print(data['processed_essays'].head())
data['processed_titles'] = data['cleaned_titles'].apply(preprocess_data)
print(data['processed_titles'].head())
data['processed_summary'] = data['cleaned_summary'].apply(preprocess_data)
print(data['processed_summary'].head())

0    student english learner working english second...
1    student arrive school eager learn polite gener...
2    true champion always one win gut mia hamm quot...
3    work unique school filled esl english second l...
4    second grade classroom next year made around k...
Name: processed_essays, dtype: object
0          educational support english learner home
1                   wanted projector hungry learner
2    soccer equipment awesome middle school student
3                             techie kindergartener
4                             interactive math tool
Name: processed_titles, dtype: object
0    studentsneedopportunitiespracticebeginningread...
1    studentsneedprojectorhelpviewingeducationalpro...
2    studentsneedshineguardsathleticsockssoccerball...
3     studentsneedengagereadingmathwayinspireminiipads
4    studentsneedhandspracticemathematicsfunpersona...
Name: processed_summary, dtype: object


In [9]:
from collections import Counter

all_text = (
    " ".join(data['processed_essays']) + " " +
    " ".join(data['processed_titles']) + " " +
    " ".join(data['processed_summary'])
)

# Count word frequencies
word_counts = Counter(all_text.split())

# Function to filter rare words
def remove_rare_words(text, min_freq=5):
    return " ".join([w for w in text.split() if word_counts[w] >= min_freq])

# Apply filtering
data['filtered_essays'] = data['processed_essays'].apply(lambda x: remove_rare_words(x, min_freq=5))
data['filtered_titles'] = data['processed_titles'].apply(lambda x: remove_rare_words(x, min_freq=5))
data['filtered_summary'] = data['processed_summary'].apply(lambda x: remove_rare_words(x, min_freq=5))

print("\nAfter Removing Low-Frequency Words:")
display(data[['filtered_essays', 'filtered_titles', 'filtered_summary']].head())


After Removing Low-Frequency Words:


,filtered_essays,filtered_titles,filtered_summary
0,student english learner working english second...,educational support english learner home,
1,student arrive school eager learn polite gener...,wanted projector hungry learner,
2,true champion always one win gut quote best de...,soccer equipment awesome middle school student,
3,work unique school filled esl english second l...,techie kindergartener,
4,second grade classroom next year made around k...,interactive math tool,


In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [11]:
# Bag of Words (BoW)
bow_vectorizer = CountVectorizer(max_features=5000)  # top 5000 words
X_bow = bow_vectorizer.fit_transform(data['filtered_essays'])

# TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(data['filtered_essays'])

# Word2Vec / embeddings-ready (tokenized lists)
data['tokens'] = data['filtered_essays'].apply(lambda x: x.split())

print("\nData is ready for embeddings!")
print("BoW Shape:", X_bow.shape)
print("TF-IDF Shape:", X_tfidf.shape)
print("Example tokens:\n", data['tokens'].head())


Data is ready for embeddings!
BoW Shape: (99469, 5000)
TF-IDF Shape: (99469, 5000)
Example tokens:
 0    [student, english, learner, working, english, ...
1    [student, arrive, school, eager, learn, polite...
2    [true, champion, always, one, win, gut, quote,...
3    [work, unique, school, filled, esl, english, s...
4    [second, grade, classroom, next, year, made, a...
Name: tokens, dtype: object


In [12]:
# Unigrams
tfidf_uni = TfidfVectorizer(max_features=5000, ngram_range=(1,1))
X_tfidf_uni = tfidf_uni.fit_transform(data['filtered_essays'])

# Bigrams
tfidf_bi = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_tfidf_bi = tfidf_bi.fit_transform(data['filtered_essays'])

# Trigrams
tfidf_tri = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X_tfidf_tri = tfidf_tri.fit_transform(data['filtered_essays'])

print("Unigram Shape:", X_tfidf_uni.shape)
print("Bigram Shape:", X_tfidf_bi.shape)
print("Trigram Shape:", X_tfidf_tri.shape)

Unigram Shape: (99469, 5000)
Bigram Shape: (99469, 5000)
Trigram Shape: (99469, 5000)


In [18]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy

In [14]:
import gensim.downloader as api
import numpy as np

# Load pre-trained Google News vectors (300d)
w2v_model = api.load("word2vec-google-news-300")

# Function: average embeddings for a document
def document_vector(doc):
    # Keep only words in vocab
    words = [w for w in doc if w in w2v_model]
    if len(words) == 0:
        return np.zeros(300)
    return np.mean(w2v_model[words], axis=0)

# Convert tokenized essays into embeddings
data['w2v_vector'] = data['tokens'].apply(document_vector)

# Stack into a matrix
X_w2v = np.vstack(data['w2v_vector'].values)

print("Word2Vec Shape:", X_w2v.shape)


KeyboardInterrupt: 

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
y = data['project_is_approved']   # 1 = approved, 0 = not approved

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))  # uni+bi-grams
X_tfidf = tfidf.fit_transform(data['processed_essays'])

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42, stratify=y
)

log_reg = LogisticRegression(max_iter=200, class_weight='balanced')
params_lr = {"C":[0.01, 0.1, 1, 10]}   # Regularization strength
grid_lr = GridSearchCV(log_reg, params_lr, cv=5, scoring='f1')
grid_lr.fit(X_train, y_train)

print("Best Logistic Regression Params:", grid_lr.best_params_)

y_pred_lr = grid_lr.predict(X_test)

svm = SVC(kernel="linear", class_weight="balanced")
params_svm = {"C":[0.1, 1, 10]}
grid_svm = GridSearchCV(svm, params_svm, cv=3, scoring='f1')
grid_svm.fit(X_train, y_train)

print("Best SVM Params:", grid_svm.best_params_)

y_pred_svm = grid_svm.predict(X_test)

def evaluate_model(y_true, y_pred, model_name):
    print(f"\n📊 {model_name} Performance:")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall   :", recall_score(y_true, y_pred))
    print("F1-score :", f1_score(y_true, y_pred))
    print("\nClassification Report:\n", classification_report(y_true, y_pred))

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=["Not Approved","Approved"],
                yticklabels=["Not Approved","Approved"])
    plt.title(f"{model_name} - Confusion Matrix")
    plt.show()
evaluate_model(y_test, y_pred_lr, "Logistic Regression (TF-IDF)")
evaluate_model(y_test, y_pred_svm, "SVM (TF-IDF)")

Best Logistic Regression Params: {'C': 1}


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

X_tfidf = tfidf.fit_transform(data['processed_essays']).toarray()


y = data['project_is_approved'].values

# Train-test split
from sklearn.model_selection import train_test_split
X_train_tfidf, X_test_tfidf, X_train_w2v, X_test_w2v, y_train, y_test = train_test_split(
    X_tfidf, X_w2v, y, test_size=0.2, random_state=42, stratify=y
)

# Input 1: TF-IDF
input_tfidf = Input(shape=(X_train_tfidf.shape[1],), name="tfidf_input")
x1 = Dense(256, activation="relu")(input_tfidf)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.3)(x1)

# Input 2: Word2Vec
input_w2v = Input(shape=(X_train_w2v.shape[1],), name="w2v_input")
x2 = Dense(128, activation="relu")(input_w2v)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.3)(x2)

# Merge the two branches
merged = Concatenate()([x1, x2])
z = Dense(128, activation="relu")(merged)
z = Dropout(0.3)(z)
output = Dense(1, activation="sigmoid")(z)

# Model
model = Model(inputs=[input_tfidf, input_w2v], outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

history = model.fit(
    [X_train_tfidf, X_train_w2v], y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

y_pred_probs = model.predict([X_test_tfidf, X_test_w2v])
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n📊 Deep Learning Model Performance:")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1-score :", f1_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Not Approved","Approved"],
            yticklabels=["Not Approved","Approved"])
plt.title("Deep Learning Model - Confusion Matrix")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, auc, precision_recall_curve
)
import seaborn as sns

def evaluate_model(name, y_true, y_pred, y_pred_probs):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"\n===== {name} =====")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1-score : {f1:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=["Not Approved","Approved"],
                yticklabels=["Not Approved","Approved"])
    plt.title(f"{name} - Confusion Matrix")
    plt.show()

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_pred_probs)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.2f})")
    plt.plot([0,1], [0,1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"{name} - ROC Curve")
    plt.legend(loc="lower right")
    plt.show()

    # Precision–Recall Curve
    precision, recall, _ = precision_recall_curve(y_true, y_pred_probs)
    plt.plot(recall, precision, label=name)
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"{name} - Precision–Recall Curve")
    plt.legend()
    plt.show()

    return acc, prec, rec, f1


In [ ]:
results = {}

# Example: Logistic Regression (TF-IDF features)
y_pred = log_reg.predict(X_test_tfidf)
y_pred_probs = log_reg.predict_proba(X_test_tfidf)[:,1]
results["LogReg_TFIDF"] = evaluate_model("Logistic Regression (TF-IDF)", y_test, y_pred, y_pred_probs)

# Example: Random Forest
y_pred = rf_model.predict(X_test_tfidf)
y_pred_probs = rf_model.predict_proba(X_test_tfidf)[:,1]
results["RandomForest_TFIDF"] = evaluate_model("Random Forest (TF-IDF)", y_test, y_pred, y_pred_probs)

# Example: SVM
y_pred = svm_model.predict(X_test_tfidf)
y_pred_probs = svm_model.decision_function(X_test_tfidf)  # SVM outputs scores
results["SVM_TFIDF"] = evaluate_model("SVM (TF-IDF)", y_test, y_pred, y_pred_probs)

# Example: XGBoost
y_pred = xgb_model.predict(X_test_tfidf)
y_pred_probs = xgb_model.predict_proba(X_test_tfidf)[:,1]
results["XGBoost_TFIDF"] = evaluate_model("XGBoost (TF-IDF)", y_test, y_pred, y_pred_probs)


In [ ]:
# Deep Learning Predictions
y_pred_probs = model.predict([X_test_tfidf, X_test_w2v]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

results["DeepLearning"] = evaluate_model("Deep Learning (TF-IDF + Word2Vec)", y_test, y_pred, y_pred_probs)


In [ ]:
import pandas as pd

metrics_df = pd.DataFrame(results, index=["Accuracy","Precision","Recall","F1-score"]).T
print("\n📊 Model Comparison:")
print(metrics_df.sort_values("F1-score", ascending=False))
